In [7]:
from keras.models import Model
from keras.layers import  Input , Conv2D , MaxPooling2D , UpSampling2D , concatenate , Conv2DTranspose , BatchNormalization
from keras.optimizers import Adam
from keras.layers import Activation , MaxPool2D ,Concatenate


In [8]:
def conv_block(input , num_filter):
  x = Conv2D(num_filter, 3 , padding = "same")(input)
  x =  BatchNormalization()(x)
  x = Activation("relu")(x)#right arrow movment 1
  x = Conv2D(num_filter, 3 , padding = "same")(x)
  x =  BatchNormalization()(x)
  x = Activation("relu")(x)#right arrow movment

  return x


In [9]:
#alternate without batchNormalization
# def conv_block(input, num_filters):
#   x = Conv2D(num_filters, 3 ,padding = "same" , Activation = "relu" )(input)
#   x = Conv2D(num_filters, 3 ,padding = "same" , Activation = "relu" )(x)
#   return x


In [14]:
def encoder_block(input , num_filters):
  s = conv_block(input , num_filters) #ouput from  last layer of a block
  p = MaxPooling2D((2,2))(s) #maxpooling to provide to next block
  return s , p

In [18]:
def decoder_block(input , skip_features , num_filters):#input : input array from previus block ,; skip _featurs , output from previous  layer of encoder block attached to decoder
  #upsampling:
  x = Conv2DTranspose(num_filters , (2,2) , strides = 2 , padding = "same")(input)#stride = 2 denotes doubling the size of input while upsampling
  #concatenating
  x = Concatenate()([x , skip_features])
   # convoluting to right
  x = conv_block(x , num_filters)
  return x

In [22]:
# Input_shape = 128
def build_model(Input_shape):
  inputs = Input(Input_shape)
  s1 , p1 = encoder_block(inputs , 64)
  s2 , p2 = encoder_block(p1 , 128)
  s3 , p3 = encoder_block(p2 , 256)
  s4 , p4 = encoder_block(p3 , 512)

  b1 = conv_block(p4 , 1024) #base layer

  d1 = decoder_block(b1 , s4 , 512)
  d2 = decoder_block(d1 , s3 , 256)
  d3 = decoder_block(d2 , s2 , 128)
  d4 = decoder_block(d3 , s1 , 64)

  #final output layer
  outputs = Conv2D(1,1 , padding = "same" , activation = "sigmoid")(d4)
  model = Model (inputs , outputs , name = "U-net")
  return model



NameError: name 'model' is not defined